In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from sklearn import datasets, linear_model
import hdbscan
import seaborn as sns
import statsmodels.api as sm
from numpy import linalg
from math import sqrt
import time
from sklearn.decomposition import PCA
from skopt import gp_minimize, dump, load
from skopt.space import Real, Integer
from skopt.plots import plot_convergence
from skopt.utils import use_named_args
%matplotlib inline

In [3]:
dataset = pd.read_csv('shopitem.csv')
Shopgroup = pd.read_csv('Shopgroup.csv')
Itemgroup = pd.read_csv('Itemgroup.csv')

In [4]:
sparsity=1-dataset.shape[0]/214200/60
sparsity

0.9669993775287893

In [5]:
Ntr=int(dataset.shape[0]*0.6)
Nv=int(dataset.shape[0]*0.8)
sim_data=dataset.values
n=214200
m=60
train=sim_data[:Ntr,:]
validation=sim_data[Ntr:Nv,:]
test=sim_data[Nv:,:]
x_train=train[:,:2]  
x_test=test[:,:2]
x_valid=validation[:,:2]
x_train=x_train.astype(int)
x_test=x_test.astype(int)
x_valid=x_valid.astype(int)
y=train[:,2]
y_test=test[:,2]
y_valid=validation[:,2]

In [6]:
#player set
ind1=[] #matched played for each user
y1=[]
#match set
ind2=[] #players in this match
y2=[]
for u in range(n):
    ind1.append(x_train[x_train[:,0]==u,1])
    y1.append(train[x_train[:,0]==u,2])
for i in range(m):
    ind2.append(x_train[x_train[:,1]==i,0])
    y2.append(train[x_train[:,1]==i,2])

In [7]:
def shrink(x,l):
    if x>l/2:
        X=x-l/2
    elif x<-l/2:
        X=x+l/2
    else:
        X=0
    return X
Vshrink= np.vectorize(shrink)

In [8]:
def mylasso(y,x,k,l,L):
    betaols=linalg.solve(x.transpose()@x+L*np.identity(k),x.transpose()@y)
    beta = Vshrink(betaols,l)
    return beta

In [9]:
def myl0(y,x,k,l,L):
    beta=linalg.solve(x.transpose()@x+L*np.identity(k),x.transpose()@y)
    beta = beta*((beta>np.median(beta))+1)
    return beta

In [ ]:
def mygsm(foo,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,matchgroup,playergroup,n,m,k,lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb): 
    inM=matchgroup.In.max()
    exM=matchgroup.Ex.max()
    inN=playergroup.In.max()
    exN=playergroup.Ex.max()
    P=np.random.normal(2,1,(n,k))
    Q=np.random.normal(2,1,(m,k))
    S=np.random.normal(2,1,(n,k))
    T=np.random.normal(2,1,(m,k))
    A=np.random.normal(2,1,(n,k))
    B=np.random.normal(2,1,(m,k))
    Pnew=np.zeros(shape=(n,k))
    Qnew=np.zeros(shape=(m,k))
    Snew=np.zeros(shape=(n,k))
    Tnew=np.zeros(shape=(m,k))
    Anew=np.zeros(shape=(n,k))
    Bnew=np.zeros(shape=(m,k))
    yhat=np.sum(np.multiply((P[x_train[:,0],:]+S[x_train[:,0],:]+A[x_train[:,0],:]),(Q[x_train[:,1],:]+T[x_train[:,1],:]+B[x_train[:,1],:])),1)
    it=1  #number of iterations
    diff=1  #improvement over last iteration
    diffQTB=1
    diffPSA=1
    while(diff>1e-5 or it<10):
        print("diff=",diff)
        diff=np.sum(np.multiply(Pnew+Snew+Anew-P-S-A,Pnew+Snew+Anew-P-S-A))/n/k+np.sum(np.multiply(Qnew+Tnew+Bnew-Q-T-B,Qnew+Tnew+Bnew-Q-T-B))/m/k
        while(diffQTB>1e-8):
            for i in range(m):
                xpsa=P[ind2[i],:]+S[ind2[i],:]+A[ind2[i],:]
                r=y2[i]-(P[ind2[i],:]+S[ind2[i],:]+A[ind2[i],:])@(T[i,:]+B[i,:])
                Qnew[i,:]=foo(r,xpsa,k,lq,Lq)

            for i in range(inM):
                I=matchgroup.matchId[matchgroup.In==i]
                I=I[I<len(ind2)]
                playerI=[]
                for j in I:
                    playerI=np.concatenate([playerI,ind2[j]]).astype("int")
                xpsa=P[playerI,:]+S[playerI,:]+A[playerI,:]
                r=[]
                for j in I:
                    R=y2[j]-(P[ind2[j],:]+S[ind2[j],:]+A[ind2[j],:])@(Qnew[j,:]+B[j,:])
                    r=np.concatenate([r,R])
                Tnew[I,:]=foo(r,xpsa,k,lt,Lt)

            for i in range(exM):
                I=matchgroup.matchId[matchgroup.Ex==i]
                I=I[I<len(ind2)]
                playerI=[]
                for j in I:
                    playerI=np.concatenate([playerI,ind2[j]]).astype("int")
                xpsa=P[playerI,:]+S[playerI,:]+A[playerI,:]
                r=[]
                for j in I:
                    R=y2[j]-(P[ind2[j],:]+S[ind2[j],:]+A[ind2[j],:])@(Qnew[j,:]+Tnew[j,:])
                    r=np.concatenate([r,R])
                Bnew[I,:]=foo(r,xpsa,k,lb,Lb)
           
            diffQTB=np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k+np.sum(np.multiply(Tnew-T,Tnew-T))/inM/k+np.sum(np.multiply(Bnew-B,Bnew-B))/exM/k
            print("diffQTB=",diffQTB)
            Q=Qnew
            T=Tnew
            B=Bnew
        while(diffPSA>1e-8):
            for i in range(n):
                xqtb=Q[ind1[i],:]+T[ind1[i],:]+B[ind1[i],:]
                r=y1[i]-(Q[ind1[i],:]+T[ind1[i],:]+B[ind1[i],:])@(S[i,:]+A[i,:])
                Pnew[i,:]=foo(r,xqtb,k,lp,Lp)

            for i in range(inN):
                I=playergroup.playerId[playergroup.In==i]
                I=I[I<len(ind1)]
                matchI=[]
                for j in I:
                    matchI=np.concatenate([matchI,ind1[j]]).astype("int")
                xqtb=Q[matchI,:]+T[matchI,:]+B[matchI,:]
                r=[]
                for j in I:
                    R=y1[j]-(Q[ind1[j],:]+T[ind1[j],:]+B[ind1[j],:])@(Pnew[j,:]+A[j,:])
                    r=np.concatenate([r,R])
                Snew[I,:]=foo(r,xqtb,k,ls,Ls)

            for i in range(exN):
                I=playergroup.playerId[playergroup.Ex==i]
                I=I[I<len(ind1)]
                matchI=[]
                for j in I:
                    matchI=np.concatenate([matchI,ind1[j]]).astype("int")
                xqtb=Q[matchI,:]+T[matchI,:]+B[matchI,:]
                r=[]
                for j in I:
                    R=y1[j]-(Q[ind1[j],:]+T[ind1[j],:]+B[ind1[j],:])@(Pnew[j,:]+Snew[j,:])
                    r=np.concatenate([r,R])
                Anew[I,:]=foo(r,xqtb,k,la,La)  
            diffPSA=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Snew-S,Snew-S))/inN/k+np.sum(np.multiply(Anew-A,Anew-A))/exN/k
            print("diffPSA=",diffPSA)
            P=Pnew
            S=Snew
            A=Anew
        it=it+1
    
    yhat_valid=np.sum(np.multiply((P[x_valid[:,0],:]+S[x_valid[:,0],:]+A[x_valid[:,0],:]),(Q[x_valid[:,1],:]+T[x_valid[:,1],:]+B[x_valid[:,1],:])),1)
    #yhat_valid=np.round(yhat_valid,decimals=4)
    RMSE=sqrt((y_valid-yhat_valid)@(y_valid-yhat_valid)/y_valid.size)
    return RMSE,yhat_valid

In [ ]:
def mygsm1(foo,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,matchgroup,playergroup,n,m,k,lp,ls,lq,lt,Lp,Ls,Lq,Lt): 
    inM=matchgroup.In.max()
    #exM=matchgroup.Ex.max()
    inN=playergroup.In.max()
    #exN=playergroup.Ex.max()
    P=np.random.normal(2,1,(n,k))
    Q=np.random.normal(2,1,(m,k))
    S=np.random.normal(2,1,(n,k))
    T=np.random.normal(2,1,(m,k))
    #A=np.random.normal(2,1,(n,k))
    #B=np.random.normal(2,1,(m,k))
    Pnew=np.zeros(shape=(n,k))
    Qnew=np.zeros(shape=(m,k))
    Snew=np.zeros(shape=(n,k))
    Tnew=np.zeros(shape=(m,k))
    #Anew=np.zeros(shape=(n,k))
    #Bnew=np.zeros(shape=(m,k))
    yhat=np.sum(np.multiply((P[x_train[:,0],:]+S[x_train[:,0],:]),(Q[x_train[:,1],:]+T[x_train[:,1],:])),1)
    it=1  #number of iterations
    diff=1  #improvement over last iteration
    diffQTB=1
    diffPSA=1
    while(diff>1e-5 or it<10):
        print("diff=",diff)
        diff=np.sum(np.multiply(Pnew+Snew-P-S,Pnew+Snew-P-S))/n/k+np.sum(np.multiply(Qnew+Tnew-Q-T,Qnew+Tnew-Q-T))/m/k
        while(diffQTB>1e-8):
            for i in range(m):
                xpsa=P[ind2[i],:]+S[ind2[i],:]
                r=y2[i]-(P[ind2[i],:]+S[ind2[i],:])@(T[i,:])
                Qnew[i,:]=foo(r,xpsa,k,lq,Lq)

            for i in range(inM):
                I=matchgroup.matchId[matchgroup.In==i]
                I=I[I<len(ind2)]
                playerI=[]
                for j in I:
                    playerI=np.concatenate([playerI,ind2[j]]).astype("int")
                xpsa=P[playerI,:]+S[playerI,:]
                r=[]
                for j in I:
                    R=y2[j]-(P[ind2[j],:]+S[ind2[j],:])@(Qnew[j,:])
                    r=np.concatenate([r,R])
                Tnew[I,:]=foo(r,xpsa,k,lt,Lt)

           
            diffQTB=np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k+np.sum(np.multiply(Tnew-T,Tnew-T))/inM/k
            print("diffQTB=",diffQTB)
            Q=Qnew
            T=Tnew
        while(diffPSA>1e-8):
            for i in range(n):
                xqtb=Q[ind1[i],:]+T[ind1[i],:]
                r=y1[i]-(Q[ind1[i],:]+T[ind1[i],:])@(S[i,:])
                Pnew[i,:]=foo(r,xqtb,k,lp,Lp)

            for i in range(inN):
                I=playergroup.playerId[playergroup.In==i]
                I=I[I<len(ind1)]
                matchI=[]
                for j in I:
                    matchI=np.concatenate([matchI,ind1[j]]).astype("int")
                xqtb=Q[matchI,:]+T[matchI,:]
                r=[]
                for j in I:
                    R=y1[j]-(Q[ind1[j],:]+T[ind1[j],:])@(Pnew[j,:])
                    r=np.concatenate([r,R])
                Snew[I,:]=foo(r,xqtb,k,ls,Ls)

            diffPSA=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Snew-S,Snew-S))/inN/k
            print("diffPSA=",diffPSA)
            P=Pnew
            S=Snew
        it=it+1
    
    yhat_valid=np.sum(np.multiply((P[x_valid[:,0],:]+S[x_valid[:,0],:]),(Q[x_valid[:,1],:]+T[x_valid[:,1],:])),1)
    #yhat_valid=np.round(yhat_valid,decimals=4)
    RMSE=sqrt((y_valid-yhat_valid)@(y_valid-yhat_valid)/y_valid.size)
    #MAE=sum(abs((y_valid-yhat_valid)))/y_valid.size
    return RMSE,yhat_valid

In [ ]:
def mygsm0(foo,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,n,m,k,lp,lq,Lp,Lq): 
    P=np.random.normal(2,1,(n,k))
    Q=np.random.normal(2,1,(m,k))
    Pnew=np.zeros(shape=(n,k))
    Qnew=np.zeros(shape=(m,k))
    yhat=np.sum(np.multiply((P[x_train[:,0],:]),(Q[x_train[:,1],:])),1)
    it=1  #number of iterations
    diff=1  #improvement over last iteration
    diffQTB=1
    diffPSA=1
    while(diff>1e-5 or it<10):
        print("diff=",diff)
        diff=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k
        while(diffQTB>1e-8):
            for i in range(m):
                xpsa=P[ind2[i],:]
                r=y2[i]
                Qnew[i,:]=foo(r,xpsa,k,lq,Lq)   
            diffQTB=np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k
            print("diffQTB=",diffQTB)
            Q=Qnew
        while(diffPSA>1e-8):
            for i in range(n):
                xqtb=Q[ind1[i],:]
                r=y1[i]
                Pnew[i,:]=foo(r,xqtb,k,lp,Lp)

            diffPSA=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Snew-S,Snew-S))/inN/k
            print("diffPSA=",diffPSA)
            P=Pnew
            S=Snew
        it=it+1
    
    yhat_valid=np.sum(np.multiply((P[x_valid[:,0],:]),(Q[x_valid[:,1],:])),1)
    #yhat_valid=np.round(yhat_valid,decimals=4)
    RMSE=sqrt((y_valid-yhat_valid)@(y_valid-yhat_valid)/y_valid.size)
    #MAE=sum(abs((y_valid-yhat_valid)))/y_valid.size
    return RMSE,yhat_valid

2-layered gssvd rmse focused k =4,...,10

In [ ]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
ls = Real(low=1e-7, high=1, prior='uniform',
                             name='ls')
la = Real(low=1e-7, high=1, prior='uniform',
                             name='la')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
lt = Real(low=1e-7, high=1, prior='uniform',
                             name='lt')
lb = Real(low=1e-7, high=1, prior='uniform',
                             name='lb')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Ls = Real(low=1e-7, high=1, prior='uniform',
                             name='Ls')
La = Real(low=1e-7, high=1, prior='uniform',
                             name='La')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
Lt = Real(low=1e-7, high=1, prior='uniform',
                             name='Lt')
Lb = Real(low=1e-6, high=1, prior='uniform',
                             name='Lb')
dimensions = [lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb]

In [ ]:
@use_named_args(dimensions=dimensions)
def Fitness(lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb):
    print('lp:', lp)
    print()
    rmse,yhat = mygsm(mylasso,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,Shopgroup,Itemgroup,
                      n,m,k,lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb)
    print()
    print("rmse:",rmse)
    print()
    return rmse

In [ ]:
start=time.time()
rmse=np.zeros(shape=(3,7))
for k in range(4,11):
    default_parameters = [1e-2,1e-2,1e-2,1e-2,1e-2,1e-2,1e-2,1e-2,1e-2,1e-2,1e-2,1e-2]
    search_result = gp_minimize(func=Fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=100,
                            x0=default_parameters)
    para=search_result.x
    rmse[0,k-4]=mygsm(mylasso,ind1,y1,ind2,y2,x_train,x_test,y,y_test,Shopgroup,Itemgroup,n,m,k,para[0],
    para[1],
    para[2],
    para[3],
    para[4],
    para[5],
    para[6],
    para[7],
    para[8],
    para[9],
    para[10],
    para[11])[0]
print(time.time()-start)


lp: 0.01

diff= 1
diffQTB= 86.45868486092154
diffQTB= 0.0
diffPSA= 112390.957957712
diffPSA= 0.0
diff= 76.70783562199773
diff= 2.575197936705771e-31
diff= 2.575197936705771e-31
diff= 2.575197936705771e-31
diff= 2.575197936705771e-31
diff= 2.575197936705771e-31
diff= 2.575197936705771e-31
diff= 2.575197936705771e-31

rmse: 2.049589042821084

lp: 0.012965947545362079

diff= 1
diffQTB= 100.84980741869461
diffQTB= 0.0
diffPSA= 130604.80093033024
diffPSA= 0.0
diff= 79.28377754643839
diff= 5.392060052299963e-32
diff= 5.392060052299963e-32
diff= 5.392060052299963e-32
diff= 5.392060052299963e-32
diff= 5.392060052299963e-32
diff= 5.392060052299963e-32
diff= 5.392060052299963e-32

rmse: 2.6032676003811006

lp: 0.011894337687589912

diff= 1
diffQTB= 85.72304769829466
diffQTB= 0.0
diffPSA= 150333.71656574696
diffPSA= 0.0
diff= 76.53898362990708
diff= 1.4041746178914448e-31
diff= 1.4041746178914448e-31
diff= 1.4041746178914448e-31
diff= 1.4041746178914448e-31
diff= 1.4041746178914448e-31
diff= 1.40

diffPSA= 103348.38217308439
diffPSA= 0.0
diff= 75.99342199731417
diff= 3.8887773223751184e-32
diff= 3.8887773223751184e-32
diff= 3.8887773223751184e-32
diff= 3.8887773223751184e-32
diff= 3.8887773223751184e-32
diff= 3.8887773223751184e-32
diff= 3.8887773223751184e-32

rmse: 2.17283623618502

lp: 0.2755829113580682

diff= 1
diffQTB= 87.8669453905586
diffQTB= 0.0
diffPSA= 132158.1279663886
diffPSA= 0.0
diff= 77.24198457877941
diff= 1.1709328150830233e-31
diff= 1.1709328150830233e-31
diff= 1.1709328150830233e-31
diff= 1.1709328150830233e-31
diff= 1.1709328150830233e-31
diff= 1.1709328150830233e-31
diff= 1.1709328150830233e-31

rmse: 2.561392312770394

lp: 1e-07

diff= 1
diffQTB= 97.30830436808958
diffQTB= 0.0
diffPSA= 153809.76435413043
diffPSA= 0.0
diff= 79.16527061195254
diff= 7.85426483005104e-32
diff= 7.85426483005104e-32
diff= 7.85426483005104e-32
diff= 7.85426483005104e-32
diff= 7.85426483005104e-32
diff= 7.85426483005104e-32
diff= 7.85426483005104e-32

rmse: 2.5861809810283476

lp:

lp: 0.872210124925189

diff= 1
diffQTB= 91.78919188689952
diffQTB= 0.0
diffPSA= 101235.32200239324
diffPSA= 0.0
diff= 78.38311005736665
diff= 1.4756442865324507e-32
diff= 1.4756442865324507e-32
diff= 1.4756442865324507e-32
diff= 1.4756442865324507e-32
diff= 1.4756442865324507e-32
diff= 1.4756442865324507e-32
diff= 1.4756442865324507e-32

rmse: 2.3186072616669176

lp: 0.051334697715645354

diff= 1
diffQTB= 88.5342620216211
diffQTB= 0.0
diffPSA= 132240.7018259165
diffPSA= 0.0
diff= 78.39968362164733
diff= 7.324309240774566e-32
diff= 7.324309240774566e-32
diff= 7.324309240774566e-32
diff= 7.324309240774566e-32
diff= 7.324309240774566e-32
diff= 7.324309240774566e-32
diff= 7.324309240774566e-32

rmse: 2.7842123188178443

lp: 0.9493851262889137

diff= 1
diffQTB= 87.41454948356603
diffQTB= 0.0
diffPSA= 105201.87469283422
diffPSA= 0.0
diff= 77.31638138584273
diff= 2.439059002030542e-32
diff= 2.439059002030542e-32
diff= 2.439059002030542e-32
diff= 2.439059002030542e-32
diff= 2.439059002030542e-

diffPSA= 583476.4054230922
diffPSA= 0.0
diff= 79.2355331102507
diff= 2.3912590931869936e-31
diff= 2.3912590931869936e-31
diff= 2.3912590931869936e-31
diff= 2.3912590931869936e-31
diff= 2.3912590931869936e-31
diff= 2.3912590931869936e-31
diff= 2.3912590931869936e-31

rmse: 2.4584302938204914

lp: 0.4172697397751714

diff= 1
diffQTB= 87.59218084449033
diffQTB= 0.0
diffPSA= 126948.56025029739
diffPSA= 0.0
diff= 77.23631332141342
diff= 4.8406510240636087e-32
diff= 4.8406510240636087e-32
diff= 4.8406510240636087e-32
diff= 4.8406510240636087e-32
diff= 4.8406510240636087e-32
diff= 4.8406510240636087e-32
diff= 4.8406510240636087e-32

rmse: 2.1738231910480974

lp: 0.3116408058849681

diff= 1
diffQTB= 88.75817548186352
diffQTB= 0.0
diffPSA= 98255.72824007104
diffPSA= 0.0
diff= 77.23253396109416
diff= 3.351210173834307e-32
diff= 3.351210173834307e-32
diff= 3.351210173834307e-32
diff= 3.351210173834307e-32
diff= 3.351210173834307e-32
diff= 3.351210173834307e-32
diff= 3.351210173834307e-32

rmse: 2

diffQTB= 0.0
diffPSA= 97283.19803144022
diffPSA= 0.0
diff= 77.50070282363032
diff= 6.994792697511842e-32
diff= 6.994792697511842e-32
diff= 6.994792697511842e-32
diff= 6.994792697511842e-32
diff= 6.994792697511842e-32
diff= 6.994792697511842e-32
diff= 6.994792697511842e-32

rmse: 2.279172409626404

lp: 0.34285525514802695

diff= 1
diffQTB= 92.62835703480434
diffQTB= 0.0
diffPSA= 142883.618202981
diffPSA= 0.0
diff= 78.223669053925
diff= 7.112814890738901e-32
diff= 7.112814890738901e-32
diff= 7.112814890738901e-32
diff= 7.112814890738901e-32
diff= 7.112814890738901e-32
diff= 7.112814890738901e-32
diff= 7.112814890738901e-32

rmse: 2.123376231901169

lp: 1.0

diff= 1
diffQTB= 89.19648353756898
diffQTB= 0.0
diffPSA= 128795.83661318834
diffPSA= 0.0
diff= 78.46907187993054
diff= 6.139614345761355e-32
diff= 6.139614345761355e-32
diff= 6.139614345761355e-32
diff= 6.139614345761355e-32
diff= 6.139614345761355e-32
diff= 6.139614345761355e-32
diff= 6.139614345761355e-32

rmse: 2.409951420069977

l

...

lp: 0.25397683592596326

diff= 1
diffQTB= 91.75754790643433
diffQTB= 0.0
diffPSA= 102847.4109547325
diffPSA= 0.0
diff= 78.23362701298174
diff= 4.7510282667203207e-32
diff= 4.7510282667203207e-32
diff= 4.7510282667203207e-32
diff= 4.7510282667203207e-32
diff= 4.7510282667203207e-32
diff= 4.7510282667203207e-32
diff= 4.7510282667203207e-32

rmse: 2.4184392319440517

lp: 0.4593564403763012

diff= 1
diffQTB= 97.17261963325171
diffQTB= 0.0
diffPSA= 103247.96986659845
diffPSA= 0.0
diff= 77.92374832840038
diff= 6.972147525710713e-32
diff= 6.972147525710713e-32
diff= 6.972147525710713e-32
diff= 6.972147525710713e-32
diff= 6.972147525710713e-32
diff= 6.972147525710713e-32
diff= 6.972147525710713e-32

rmse: 2.3895727606451573

lp: 0.23068422170803615

diff= 1
diffQTB= 87.63982207958617
diffQTB= 0.0
diffPSA= 93202.23328863655
diffPSA= 0.0
diff= 79.41188627800346
diff= 3.5407710536711e-32
diff= 3.5407710536711e-32
diff= 3.5407710536711e-32
diff= 3.5407710536711e-32
diff= 3.5407710536711e-32
diff= 

diffPSA= 0.0
diff= 76.63581617645224
diff= 2.860919901709743e-31
diff= 2.860919901709743e-31
diff= 2.860919901709743e-31
diff= 2.860919901709743e-31
diff= 2.860919901709743e-31
diff= 2.860919901709743e-31
diff= 2.860919901709743e-31

rmse: 2.2370605649346547

lp: 0.625210802084159

diff= 1
diffQTB= 82.19017137079325
diffQTB= 0.0
diffPSA= 118636.01334081806
diffPSA= 0.0
diff= 76.31345250304992
diff= 5.474040084925921e-32
diff= 5.474040084925921e-32
diff= 5.474040084925921e-32
diff= 5.474040084925921e-32
diff= 5.474040084925921e-32
diff= 5.474040084925921e-32
diff= 5.474040084925921e-32

rmse: 2.274585475683409

lp: 0.2891877294479026

diff= 1
diffQTB= 86.61022255421327
diffQTB= 0.0
diffPSA= 107305.33459926293
diffPSA= 0.0
diff= 78.35321084175519
diff= 2.504750681886477e-32
diff= 2.504750681886477e-32
diff= 2.504750681886477e-32
diff= 2.504750681886477e-32
diff= 2.504750681886477e-32
diff= 2.504750681886477e-32
diff= 2.504750681886477e-32

rmse: 2.265764904902671

lp: 0.5404721315833185


diff= 6.2682098467374e-32

rmse: 2.352228023196356

lp: 0.9243701224113189

diff= 1
diffQTB= 87.01719877448811
diffQTB= 0.0
diffPSA= 117726.15978385253
diffPSA= 0.0
diff= 77.7879256804346
diff= 6.838710846890713e-32
diff= 6.838710846890713e-32
diff= 6.838710846890713e-32
diff= 6.838710846890713e-32
diff= 6.838710846890713e-32
diff= 6.838710846890713e-32
diff= 6.838710846890713e-32

rmse: 2.1805894725444617

lp: 0.9695225151146584

diff= 1
diffQTB= 88.81176520708055
diffQTB= 0.0
diffPSA= 88246.32530940662
diffPSA= 0.0
diff= 79.5283645937836
diff= 2.436164644275457e-31
diff= 2.436164644275457e-31
diff= 2.436164644275457e-31
diff= 2.436164644275457e-31
diff= 2.436164644275457e-31
diff= 2.436164644275457e-31
diff= 2.436164644275457e-31

rmse: 6.797020208504627

lp: 1e-07

diff= 1
diffQTB= 91.7341101421943
diffQTB= 0.0
diffPSA= 112306.17766964737
diffPSA= 0.0
diff= 80.14286452053054
diff= 5.370447147353811e-32
diff= 5.370447147353811e-32
diff= 5.370447147353811e-32
diff= 5.370447147353811e-

diff= 4.6321876026553243e-32

rmse: 2.532827358331641

lp: 0.23492368896434745

diff= 1
diffQTB= 89.36150600041266
diffQTB= 0.0
diffPSA= 120065.92911902256
diffPSA= 0.0
diff= 78.24810859821545
diff= 5.125596576401692e-32
diff= 5.125596576401692e-32
diff= 5.125596576401692e-32
diff= 5.125596576401692e-32
diff= 5.125596576401692e-32
diff= 5.125596576401692e-32
diff= 5.125596576401692e-32

rmse: 2.66225807036607

lp: 0.6755956615219402

diff= 1
diffQTB= 86.88665178366779
diffQTB= 0.0
diffPSA= 96145.21949596981
diffPSA= 0.0
diff= 78.83430497450459
diff= 1.068345837168802e-31
diff= 1.068345837168802e-31
diff= 1.068345837168802e-31
diff= 1.068345837168802e-31
diff= 1.068345837168802e-31
diff= 1.068345837168802e-31
diff= 1.068345837168802e-31

rmse: 2.569838616199671

lp: 0.7478700291091787

diff= 1
diffQTB= 85.19339675834867
diffQTB= 0.0
diffPSA= 155947.5973572211
diffPSA= 0.0
diff= 77.38559700141872
diff= 7.999732512620387e-32
diff= 7.999732512620387e-32
diff= 7.999732512620387e-32
diff= 7.

diff= 7.164801087053961e-32

rmse: 2.888217739084221

lp: 0.18615660095200254

diff= 1
diffQTB= 94.21455857764767
diffQTB= 0.0
diffPSA= 224115.22200502682
diffPSA= 0.0
diff= 76.47072978946092
diff= 8.493764858663199e-32
diff= 8.493764858663199e-32
diff= 8.493764858663199e-32
diff= 8.493764858663199e-32
diff= 8.493764858663199e-32
diff= 8.493764858663199e-32
diff= 8.493764858663199e-32

rmse: 2.8321444555412176

lp: 1.0

diff= 1
diffQTB= 88.91629292470408
diffQTB= 0.0
diffPSA= 113866.0201820109
diffPSA= 0.0
diff= 77.9317868810096
diff= 6.08136507530517e-32
diff= 6.08136507530517e-32
diff= 6.08136507530517e-32
diff= 6.08136507530517e-32
diff= 6.08136507530517e-32
diff= 6.08136507530517e-32
diff= 6.08136507530517e-32

rmse: 2.306538891034273

lp: 0.9031612342413375

diff= 1
diffQTB= 91.20880809585935
diffQTB= 0.0
diffPSA= 94663.63438940504
diffPSA= 0.0
diff= 78.96306054699716
diff= 6.943407129444621e-32
diff= 6.943407129444621e-32
diff= 6.943407129444621e-32
diff= 6.943407129444621e-32
di

1-layered gssvd rmse focused 

In [ ]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
ls = Real(low=1e-7, high=1, prior='uniform',
                             name='ls')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
lt = Real(low=1e-7, high=1, prior='uniform',
                             name='lt')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Ls = Real(low=1e-7, high=1, prior='uniform',
                             name='Ls')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
Lt = Real(low=1e-7, high=1, prior='uniform',
                             name='Lt')
dimensions = [lp,ls,lq,lt,Lp,Ls,Lq,Lt]

In [ ]:
@use_named_args(dimensions=dimensions)
def Fitness(lp,ls,lq,lt,Lp,Ls,Lq,Lt):
    print('lp:', lp)
    print()
    rmse,yhat = mygsm1(mylasso,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,Shopgroup,Itemgroup,
                      n,m,k,lp,ls,lq,lt,Lp,Ls,Lq,Lt)
    print()
    print("rmse:",rmse)
    print()
    return rmse

In [ ]:
start=time.time()
for k in range(4,11):
    default_parameters = [1e-2,1e-2,1e-2,1e-2,1e-2,1e-2,1e-2,1e-2]
    search_result = gp_minimize(func=Fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=100,
                            x0=default_parameters)
    para=search_result.x
    rmse[1,k-4]=mygsm1(mylasso,ind1,y1,ind2,y2,x_train,x_test,y,y_test,Shopgroup,Itemgroup,n,m,k,para[0],
    para[1],
    para[2],
    para[3],
    para[4],
    para[5],
    para[6],
    para[7])[0]
print(time.time()-start)

nogroupsvd rmse focused

In [ ]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
dimensions = [lp,lq,Lp,Lq]

In [ ]:
@use_named_args(dimensions=dimensions)
def Fitness(lp,lq,Lp,Lq):
    print('lp:', lp)
    print()
    rmse,yhat = mygsm0(mylasso,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,
                      n,m,k,lp,lq,Lp,Lq)
    print()
    print("rmse:",rmse)
    print()
    return rmse

In [ ]:
start=time.time()
for k in range(4,11):
    default_parameters = [1e-2,1e-2,1e-2,1e-2]
    search_result = gp_minimize(func=Fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=100,
                            x0=default_parameters)
    para=search_result.x
    rmse[2,k-4]=mygsm0(mylasso,ind1,y1,ind2,y2,x_train,x_test,y,y_test,n,m,k,para[0],
    para[1],
    para[2],
    para[3])[0]
print(time.time()-start)

2-layered gssvd recommender focused

In [ ]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
ls = Real(low=1e-7, high=1, prior='uniform',
                             name='ls')
la = Real(low=1e-7, high=1, prior='uniform',
                             name='la')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
lt = Real(low=1e-7, high=1, prior='uniform',
                             name='lt')
lb = Real(low=1e-7, high=1, prior='uniform',
                             name='lb')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Ls = Real(low=1e-7, high=1, prior='uniform',
                             name='Ls')
La = Real(low=1e-7, high=1, prior='uniform',
                             name='La')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
Lt = Real(low=1e-7, high=1, prior='uniform',
                             name='Lt')
Lb = Real(low=1e-6, high=1, prior='uniform',
                             name='Lb')
dimensions = [lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb]

In [ ]:
@use_named_args(dimensions=dimensions)
def fitness(lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb):
    Y = mygsm(myl0,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,Shopgroup,Itemgroup,
                      n,m,k,lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb)[1]
    b=y_valid>(np.median(y_valid))
    acc=[]
    for i in range(1,10):
        a=Y>=(Y[np.argsort(Y)[-i]])
        acc=np.append(acc,sum(b[a])/len(b[a]))
    return acc.mean()

In [ ]:
start=time.time()
accuracy=np.zeros(shape=(3,7))
for k in range(4,11):
    default_parameters = [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5]
    search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=100,
                            x0=default_parameters)
    para=search_result.x
    
    a=mygsm(myl0,ind1,y1,ind2,y2,x_train,x_test,y,y_test,Shopgroup,Itemgroup,n,m,k,para[0],
    para[1],
    para[2],
    para[3],
    para[4],
    para[5],
    para[6],
    para[7],
    para[8],
    para[9],
    para[10],
    para[11])
    b=y_valid>(np.median(y_valid))
    acc=[]
    for i in range(1,10):
        a=Y>=(Y[np.argsort(Y)[-i]])
        acc=np.append(acc,sum(b[a])/len(b[a]))
    accuracy[0,k-4]=acc.mean()
print(time.time()-start)

1-layered gssvd recommender focused

In [ ]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
ls = Real(low=1e-7, high=1, prior='uniform',
                             name='ls')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
lt = Real(low=1e-7, high=1, prior='uniform',
                             name='lt')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Ls = Real(low=1e-7, high=1, prior='uniform',
                             name='Ls')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
Lt = Real(low=1e-7, high=1, prior='uniform',
                             name='Lt')
dimensions = [lp,ls,lq,lt,Lp,Ls,Lq,Lt]

In [ ]:
@use_named_args(dimensions=dimensions)
def fitness(lp,ls,lq,lt,Lp,Ls,Lq,Lt):
    Y = mygsm1(myl0,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,Shopgroup,Itemgroup,
                      n,m,k,lp,ls,lq,lt,Lp,Ls,Lq,Lt)[1]
    b=y_valid>(np.median(y_valid))
    acc=[]
    for i in range(1,10):
        a=Y>=(Y[np.argsort(Y)[-i]])
        acc=np.append(acc,sum(b[a])/len(b[a]))
    return acc.mean()

In [ ]:
start=time.time()
for k in range(4,11):
    default_parameters = [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5]
    search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=100,
                            x0=default_parameters)
    para=search_result.x
    
    a=mygsm1(myl0,ind1,y1,ind2,y2,x_train,x_test,y,y_test,Shopgroup,Itemgroup,n,m,k,para[0],
    para[1],
    para[2],
    para[3],
    para[4],
    para[5],
    para[6],
    para[7])
    b=y_valid>(np.median(y_valid))
    acc=[]
    for i in range(1,10):
        a=Y>=(Y[np.argsort(Y)[-i]])
        acc=np.append(acc,sum(b[a])/len(b[a]))
    accuracy[1,k-4]=acc.mean()
print(time.time()-start)

svd recommender focused

In [ ]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
dimensions = [lp,lq,Lp,Lq]

In [ ]:
@use_named_args(dimensions=dimensions)
def fitness(lp,lq,Lp,Lq):
    Y = mygsm0(myl0,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,
                      n,m,k,lp,lq,Lp,Lq)[1]
    b=y_valid>(np.median(y_valid))
    acc=[]
    for i in range(1,10):
        a=Y>=(Y[np.argsort(Y)[-i]])
        acc=np.append(acc,sum(b[a])/len(b[a]))
    return acc.mean()

In [ ]:
start=time.time()
for k in range(4,11):
    default_parameters = [0.5,0.5,0.5,0.5]
    search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=100,
                            x0=default_parameters)
    para=search_result.x
    
    a=mygsm0(myl0,ind1,y1,ind2,y2,x_train,x_test,y,y_test,n,m,k,para[0],
    para[1],
    para[2],
    para[3])
    b=y_valid>(np.median(y_valid))
    acc=[]
    for i in range(1,10):
        a=Y>=(Y[np.argsort(Y)[-i]])
        acc=np.append(acc,sum(b[a])/len(b[a]))
    accuracy[2,k-4]=acc.mean()
print(time.time()-start)